This is code for creating simple plots of daily Fitbit data in python. 

It will create the following plots:
* Time Series w/ rolling mean
* Histogram
* Density Plot parsed by day of the week 

Taken from [Bonnie Barrilleaux](http://bonniebarrilleaux.com/). 
Her Fitbit data post is avaiable here: [Freeing My Fitbit Data](http://bonniebarrilleaux.com/2013/12/22/freeing-my-fitbit-data/)
This code was copied from her [gist here](https://gist.github.com/bbarrilleaux/8090081#file-fitbitstats-py)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import pylab
from matplotlib import pyplot
from scipy.stats import gaussian_kde
 
data = pd.read_csv('fitbit.csv', usecols = [0, 1], parse_dates = True)
 
#count and remove days where <100 steps were logged, 
#because those were when Fitbit wasn't working.
data[data['steps'] <= 100].count()
data = data[data['steps'] > 100]
 
data['daynumber'] = data.date.map(lambda x: pd.to_datetime(x).weekday()) 
data['weekday'] = data.date.map(lambda x: pd.to_datetime(x).strftime("%a")) 
data['rollingmean'] = pd.rolling_mean(data['steps'], window = 50, min_periods = 0)
 
 
#Time series plot with rolling mean
pyplot.plot(pd.to_datetime(data['date']), data['steps'], "r", linewidth = 1)
pyplot.plot(pd.to_datetime(data['date']), data['rollingmean'], "b", linewidth = 2)
pyplot.xticks(rotation = 90)
pyplot.title('Fitbit steps per day', size = 24)
pyplot.ylabel('Steps', size = 18)
pyplot.subplots_adjust(bottom = .15, top = .9)
pyplot.show()
 
#histogram of steps
pyplot.hist(data['steps'], bins = 50, histtype = 'stepfilled')
pyplot.title('Histogram of Fitbit steps', size = 24)
pyplot.xlabel('Steps', size = 18)
pyplot.ylabel('Number of days', size = 18)
pyplot.show()
 
 
# Density plot broken into days of the week
sorteddata = data.sort_index(by = ['daynumber'], ascending = [True])
 
for key, grp in sorteddata.groupby(['weekday']):
	print(key)
	print('Average number of steps:', grp['steps'].mean())
	print('Percent of days exceeding 15,000 steps:', (grp[grp['steps'] > 15000].count()[1])*100/grp.count()[1], "%")
	density = gaussian_kde(grp['steps'])
	xnew = np.linspace(0,30000,200)
	density.covariance_factor = lambda : .25
	density._compute_covariance()
	pyplot.plot(xnew, density(xnew), label = key, linewidth = 2)
 
pyplot.legend(fancybox = True, fontsize = 18)
pyplot.title('Histogram of Fitbit steps by weekday', size = 24)
pyplot.xlabel('Steps', size = 18)
pyplot.ylabel('Density', size = 18)
pyplot.subplots_adjust(left = .15, right = .9)
pyplot.show()